# Exploring Betfair API's

### Create session

In [1]:
import json
import pandas as pd

In [ ]:
import requests
  
application_key = #Application Key
 
payload = #Username and password
headers = {'X-Application': 'test', 'Content-Type': 'application/x-www-form-urlencoded'}
 
resp = requests.post('https://identitysso-cert.betfair.com/api/certlogin', data=payload, cert=('Betfair.crt', 'client-2048.pem'), headers=headers)
 
resp_json = resp.json()
session_token = resp_json['sessionToken']

if resp.status_code == 200:
    print(resp_json['loginStatus'])
    print(session_token)

else:
    print("Request failed.")

### Listing events

eventTypeIds = sport, football = 1

In [8]:
eventId = "32286072"

In [10]:
req = [
    {
        "jsonrpc": "2.0",
        "method": "SportsAPING/v1.0/listEvents",
        "params": {
            "filter": {
                "eventTypeIds": ["1"],
                #"marketCountries": ["GB"],
                #"eventIds": [eventId]
                #"inPlayOnly": "False"
            }
        },
        "id": 1
    }
]

In [ ]:
application_key = 'nCIUYW9B6gb101Vi'

headers = {'X-Application': application_key, 
           'X-Authentication': session_token,
           'Accept': 'application/json'
           }


url="https://api.betfair.com/exchange/betting/json-rpc/v1"

jsonrpc_req = json.dumps(req[0])
 
response = requests.post(url, data=jsonrpc_req, headers=headers)
 
print(json.dumps(json.loads(response.text), indent=3))

### Listing markets

In [15]:
market_req = [
    {
        "jsonrpc": "2.0",
        "method": "SportsAPING/v1.0/listMarketCatalogue",
        "params": {
            "filter": {
                "eventIds": [eventId],
            },
            "maxResults": "200",
            "marketProjection": [
                "COMPETITION",
                "EVENT",
                "EVENT_TYPE",
                "RUNNER_DESCRIPTION",
                "RUNNER_METADATA",
                "MARKET_START_TIME"
            ]
        },
        "id": 1
    }
]

In [16]:
application_key = #Application key

headers = {'X-Application': application_key, 
           'X-Authentication': session_token,
           'Accept': 'application/json'
           }


url="https://api.betfair.com/exchange/betting/json-rpc/v1"

jsonrpc_req = json.dumps(market_req[0])
 
market_response = requests.post(url, data=jsonrpc_req, headers=headers)
 
print(json.dumps(json.loads(market_response.text), indent=3))

{
   "jsonrpc": "2.0",
   "result": [],
   "id": 1
}


In [18]:
def get_odds(market_id):
    '''Return odds of all runners in a market'''
    
    odds_req = [{
        "jsonrpc": "2.0",
        "method": "SportsAPING/v1.0/listMarketBook",
        "params": {
            "marketIds": [market_id],
            "priceProjection": {
                "priceData": ["EX_BEST_OFFERS", "EX_TRADED"],
                "virtualise": "true"
            }
        },
        "id": 1
    }]
    
    application_key = 'nCIUYW9B6gb101Vi'

    headers = {'X-Application': application_key, 
               'X-Authentication': session_token,
               'Accept': 'application/json'
               }


    url="https://api.betfair.com/exchange/betting/json-rpc/v1"

    jsonrpc_req = json.dumps(odds_req[0])

    odds_response = requests.post(url, data=jsonrpc_req, headers=headers)
    
    odds_json = json.loads(odds_response.text)
    
    return odds_json['result'][0]

### Get market price

In [21]:
odds_req = [{
    "jsonrpc": "2.0",
    "method": "SportsAPING/v1.0/listMarketBook",
    "params": {
        "marketIds": ["1.213708836"],
        "priceProjection": {
            "priceData": ["EX_BEST_OFFERS", "EX_TRADED"],
            "virtualise": "true"
        }
    },
    "id": 1
}]

Info needed for odds:
- marketIds
- selectionIds

In [22]:
application_key = 'nCIUYW9B6gb101Vi'

headers = {'X-Application': application_key, 
           'X-Authentication': session_token,
           'Accept': 'application/json'
           }


url="https://api.betfair.com/exchange/betting/json-rpc/v1"

jsonrpc_req = json.dumps(odds_req[0])
 
odds_response = requests.post(url, data=jsonrpc_req, headers=headers)
 
print(json.dumps(json.loads(odds_response.text), indent=3))

{
   "jsonrpc": "2.0",
   "result": [
      {
         "marketId": "1.213708836",
         "isMarketDataDelayed": false,
         "status": "CLOSED",
         "betDelay": 5,
         "bspReconciled": false,
         "complete": true,
         "inplay": false,
         "numberOfWinners": 1,
         "numberOfRunners": 2,
         "numberOfActiveRunners": 0,
         "totalMatched": 0.0,
         "totalAvailable": 0.0,
         "crossMatching": false,
         "runnersVoidable": false,
         "version": 5220955521,
         "runners": [
            {
               "selectionId": 47972,
               "handicap": 0.0,
               "status": "LOSER",
               "ex": {
                  "availableToBack": [],
                  "availableToLay": [],
                  "tradedVolume": []
               }
            },
            {
               "selectionId": 47973,
               "handicap": 0.0,
               "status": "WINNER",
               "ex": {
                  "availab

In [20]:
def get_largest_market(all_odds):
    '''Return price of largest market size'''
    
    sizes = []
    if len(all_odds) == 0:
        price = 'null'
    else:     
        for i in range(0, len(all_odds)):
            sizes.append(all_odds[i]['size'])
        max_idx = sizes.index(max(sizes))
        price = all_odds[max_idx]['price']

    return price

In [21]:
def get_risk(price):
    '''Return the risk level'''
    
    if price == 'null':
        risk = 'NA'
    else:
        if price > 1 and price < 1.3:
            risk = 'Low'
        elif price >= 1.3 and price < 1.6:
            risk = 'Medium'
        elif price >= 1.6 and price < 1.9:
            risk = 'High'
        else:
            risk = 'Too high'
        
    return risk

In [23]:
num_markets = len(json.loads(market_response.text)['result'])
runners = []

for i in range(0, num_markets):
    market_id = json.loads(market_response.text)['result'][i]['marketId']
    market_name = json.loads(market_response.text)['result'][i]['marketName']
    
    num_runners = len(json.loads(market_response.text)['result'][i]['runners']) 
    market_odds = get_odds(market_id)
   
    for j in range(0, num_runners):
        runnerName = json.loads(market_response.text)['result'][i]['runners'][j]['runnerName']
        runnerId = json.loads(market_response.text)['result'][i]['runners'][j]['selectionId']
        selectionId = market_odds['runners'][j]['selectionId']
        handicap = market_odds['runners'][j]['handicap']
        
        if selectionId == runnerId:
            all_odds = market_odds['runners'][j]['ex']['availableToBack']
            price = get_largest_market(all_odds)
            risk = get_risk(price)
        else:
            raise Exception('RunnerId and SelectionId mismatch')
                
        runner = {'marketId': market_id, 'marketName': market_name, 'runnerName': runnerName, 'handicap': handicap, 'runnerId': runnerId, 'runnerOdds': price, 'risk': risk}
        runners.append(runner)

In [24]:
runners_df = pd.DataFrame(runners)
runners_df.head(10)

""


### Low risk markets

In [25]:
low_risk_df = runners_df[runners_df['risk'] == 'Low']
print(len(low_risk_df))
low_risk_df.head(20)

100


,marketId,marketName,runnerName,handicap,runnerId,runnerOdds,risk
3,1.213152820,Over/Under 4.5 Goals,Under 4.5 Goals,0.0,1222347,1.19,Low
5,1.213152889,Over/Under 5.5 Goals,Under 5.5 Goals,0.0,1485567,1.06,Low
63,1.213152887,Over/Under 6.5 Goals,Under 6.5 Goals,0.0,2542448,1.01,Low
77,1.213152885,Over/Under 0.5 Goals,Over 0.5 Goals,0.0,5851483,1.05,Low
90,1.213152878,First Half Goals 2.5,Under 2.5 Goals,0.0,47972,1.14,Low
93,1.213152875,Man City Win to Nil,No,0.0,110503,1.22,Low
94,1.213152876,Over/Under 7.5 Goals,Under 7.5 Goals,0.0,1485572,1.01,Low
96,1.213152873,Man City +3,Man City +3,0.0,6563232,1.03,Low
100,1.213152874,Real Madrid Win to Nil,No,0.0,110503,1.14,Low
141,1.213152871,Real Madrid +3,Real Madrid +3,0.0,6582713,1.08,Low


In [28]:
#Store current odds to reflect on after the event has happened
runners_df.to_csv('Games/ManCity-vs-RealMadrid-Runners.csv', index = False)